# Google Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# install

In [2]:
!pip3 install transformers peft

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 5.8 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manyl

# 7.2.1 LoRA

### LoRA 실습

In [3]:
from transformers import AutoModelForCausalLM, AutoConfig
from peft import get_peft_model, LoraConfig, TaskType

# 모델 로드
model = AutoModelForCausalLM.from_pretrained("nlpai-lab/ko-gemma-2b-v1")

# peft
peft_config = LoraConfig(
    task_type=TaskType.SEQ_2_SEQ_LM,
    inference_mode=False,
    r=8,
    lora_alpha=32,
    lora_dropout=0.1,
)
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

# 모델 저장
model.save_pretrained("/content/drive/MyDrive/Books/outputs/peft")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/676 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/13.5k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

trainable params: 921,600 || all params: 2,507,094,016 || trainable%: 0.036759690467068624


In [4]:
!ls /content/drive/MyDrive/Books/outputs/peft
!du -sh /content/drive/MyDrive/Books/outputs/peft

adapter_config.json  adapter_model.safetensors	README.md
3.6M	/content/drive/MyDrive/Books/outputs/peft


### LoRA 모델 추론

In [3]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig
from peft import PeftModel, PeftConfig

peft_model_id = "/content/drive/MyDrive/Books/outputs/peft"
config = PeftConfig.from_pretrained(peft_model_id)
model = AutoModelForCausalLM.from_pretrained(config.base_model_name_or_path)
model = PeftModel.from_pretrained(model, peft_model_id)
model = model.to('cuda')
model.eval()

tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)
inputs = tokenizer.apply_chat_template(
    [
        {"role": "user", "content": "대한민국 국보 제 1호는 무엇입니까?"},
    ],
    add_generation_prompt=True,
    return_tensors="pt",
)
gen_cfg = GenerationConfig(
    max_new_tokens=2048,
    do_sample=True,
    top_k=50,
    top_p=0.92,
    return_full_text=False,
)
with torch.no_grad():
    outputs = model.generate(input_ids=inputs.to('cuda'), generation_config=gen_cfg)
    print(tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0])

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

user
대한민국 국보 제 1호는 무엇입니까?
model
대한민국 국보 제1호는 유네스코의 지구 보전 및 경관 보호 프로그램(GPB)으로 2020년에 국제유산위원회에 제출되었습니다. 이 국보는 한국 동부 해안부에 위치한 울상동과 울상 산지 지역의 울상 섬에 대한 보존을 조직하는 것을 목표로 합니다.울상 섬은 한국 동해안에 위치해 있으며, 울상 섬과 해안의 일부가 울상 섬 국립 야생동물 보호구역이 되고 있습니다. 한국식민지 시대와 이후부터 섬을 사용해온 무역 및 어업과 함께 인간 활동으로 인한 피해가 발생했습니다. 이제 한국식민지 시대의 울상산과 울상호와 울상 지역의 특정 구역에 대한 보호가 우선시되고 있습니다.


### LoRA 미세조정